Here we show how to use our models.

In [5]:
import pandas as pd
import matplotlib.pyplot as plt
import os
import time
import sys

# Add the relative path to the system path
sys.path.append(os.path.abspath("../train_OpenSTL"))

from PIL import Image

import torch as th
from torch.utils.data import Dataset
from torch import nn

from ConvLSTM_model import ConvLSTM_Model

In [ ]:
batch_size = 16
num_layers = 2
num_hidden = [32, 64] 
custom_model_config = {
    'in_shape': [5, 3, 256, 256], # T, C, H, W
    'patch_size': 1,
    'filter_size': 1, # given to ConvLSTMCell
    'stride': 1, # given to ConvLSTMCell
    'layer_norm' : False, # given to ConvLSTMCell
    # the sum of pre_seq_length and aft_seq_length has to be = len(inputs)
    'pre_seq_length': 3,
    'aft_seq_length': 2,
    'target_seq_length': 5,
    'reverse_scheduled_sampling': 0,
    'batch_size': batch_size
}

if th.cuda.is_available():
    print("CUDA is available!")
else:
    print("CUDA is not available.")

device = th.device("cuda" if th.cuda.is_available() else "cpu")
th.cuda.empty_cache()

# Instantiate the model
input_dim = 3  # Assuming x_train shape is (batch_size, sequence_length, channels, height, width)
model = ConvLSTM_Model(num_layers, num_hidden, custom_model_config)

# Load the model
model.load_state_dict(th.load('../models/model.pth'))

In [7]:
# List of frames as PIL Image objects
frames = [Image.open('1.JPG'), Image.open('2.JPG'), Image.open('3.JPG'), Image.open('4.JPG')]
# Duration sets the display time for each frame in milliseconds
# Loop sets the number of loops. Default is 0 and means infinite
frames[0].save('output.gif', save_all=True, append_images=frames[1:], duration=1, loop=5)

In [ ]:
# how to display the gif
from IPython.display import Image
Image(filename='output.gif')

In [ ]:
# how to dispaly 2 gifs horizontally aligned
from IPython.display import display
from PIL import Image


In [ ]:
from IPython.display import HTML

# Display two GIFs horizontally aligned with titles on top of each
HTML('''
    <div style="display: flex; justify-content: space-around; align-items: center;">
        <div style="text-align: center;">
            <h3>True Sequence</h3>
            <img src="output.gif" width="300px">
        </div>
        <div style="text-align: center;">
            <h3>Predicted Sequence</h3>
            <img src="output.gif" width="300px">
        </div>
    </div>
''')
